<h1 align=center>Applied Data Science Capstone - IBM Professional Certificate  </h1>  
<h3 align=center>Research on effects of different types on venues on the price of property in the neighborhoods of Toronto  </h3>

0. Installing all required packages and modules

In [1]:
#First of all we install and import all necessary packages
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes
import requests
import pandas as pd
import numpy as np
import json
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
import seaborn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



<strong>1.Processing of datasets and data preparation </strong>  
*1.1 The first dataset*

In [2]:
#first of all we start with the processing and uploading of file from Toronro administration site. We need to extract two
#two columns - house prices and names of neighborhood 
df_eco = pd.read_csv('dataeco.csv')
df_eco.head()

,Unnamed: 0,Unnamed: 1,Economics,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Neighbourhood,Neighbourhood Id,Businesses,Child Care Spaces,Debt Risk Score,Home Prices,Local Employment,Social Assistance Recipients
1,West Humber-Clairville,1,2463,195,719,317508,58271,2912
2,Mount Olive-Silverstone-Jamestown,2,271,60,687,251119,3244,6561
3,Thistletown-Beaumond Heights,3,217,25,718,414216,1311,1276
4,Rexdale-Kipling,4,144,75,721,392271,1178,1323


In [3]:
## We quickly fix the data removing row with column names and assigning right names to columns
df_eco.columns = df_eco.loc[0]
df_eco.drop(0,inplace=True)
df_eco.head()

,Neighbourhood,Neighbourhood Id,Businesses,Child Care Spaces,Debt Risk Score,Home Prices,Local Employment,Social Assistance Recipients
1,West Humber-Clairville,1,2463,195,719,317508,58271,2912
2,Mount Olive-Silverstone-Jamestown,2,271,60,687,251119,3244,6561
3,Thistletown-Beaumond Heights,3,217,25,718,414216,1311,1276
4,Rexdale-Kipling,4,144,75,721,392271,1178,1323
5,Elms-Old Rexdale,5,67,60,692,233832,903,1683


In [4]:
df_eco = df_eco[['Neighbourhood','Home Prices']]

Next we need to find corresponding values of longitude and latitude 

*2.2 Processing of the second dataframe*

In [5]:
df_loc = pd.read_csv('Neighbourhoods.csv')
df_loc.shape
# We need only three columns out of this list: name of area, longitude and latitude
df_loc = df_loc[['AREA_NAME','LONGITUDE','LATITUDE']]
df_loc

,AREA_NAME,LONGITUDE,LATITUDE
0,Wychwood (94),-79.425515,43.676919
1,Yonge-Eglinton (100),-79.403590,43.704689
2,Yonge-St.Clair (97),-79.397871,43.687859
3,York University Heights (27),-79.488883,43.765736
4,Yorkdale-Glen Park (31),-79.457108,43.714672
...,...,...,...
135,Kennedy Park (124),-79.260382,43.725556
136,Kensington-Chinatown (78),-79.397240,43.653554
137,Kingsview Village-The Westway (6),-79.547863,43.698993
138,Kingsway South (15),-79.510577,43.653520


In [6]:
#We check the number of neighborhoods in both tables
shp1 = df_eco.shape
shp2 = df_loc.shape
print('The column with economical data consists of ' + str(shp1[0])+ ' neighborhoods')
print('The column with geographical data consists of ' + str(shp2[0])+ ' neighborhoods')
#From this we can see, that our data is correct. Now we can sort and merge it.

The column with economical data consists of 140 neighborhoods
The column with geographical data consists of 140 neighborhoods


In [7]:
#Then we sort values in both tables by the names of neighborhoods
df_eco.sort_values(['Neighbourhood'],inplace=True)
df_loc.sort_values(['AREA_NAME'],inplace=True)

In [8]:
#As we need to get geographical coordinates of those hoods in order to use Foursquare API to get info about venues
#We merge both tables
df_fin = df_eco
df_fin['Longitude'] = df_loc['LONGITUDE']
df_fin['Latitude'] = df_loc['LATITUDE']
#Let's take a look what we've got
df_fin.head()

,Neighbourhood,Home Prices,Longitude,Latitude
129,Agincourt North,375307,-79.542367,43.743430
128,Agincourt South-Malvern West,332710,-79.556175,43.758920
20,Alderwood,504233,-79.587259,43.746868
95,Annex,993491,-79.408007,43.681852
42,Banbury-Don Mills,613647,-79.442992,43.646123


In [9]:
# Visualisation of Neighborhoods
## First of all we find coordinates of Toronto on map in order to center map around it
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
loc = geolocator.geocode(address)
la = loc.latitude
lo = loc.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(la, lo))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
#We also remove all "nan" values from the table
#nan = float('nan')
#df_fin.drop(df_fin[df_fin['Latitude']==nan].index,inplace=True)
#df_fin.drop(df_fin[df_fin['Longitude']==nan].index,inplace=True)
df_fin.dropna(axis='rows',how='any',inplace=True)

In [11]:
# We create Folium map and center it around Toronto
toronto_map = folium.Map(location=[la, lo], zoom_start=10)
# Then we add points corresponding to neighbourhoods
for lat, lng, neighborhood in zip(df_fin['Latitude'], df_fin['Longitude'], df_fin['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='#000080',
        fill=True,
        fill_color='#FFFFFF',
        fill_opacity=0.9,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

*1.3 Using Foursquare API to get information about venues*

In [15]:
 #Entering Foursquare creditinals
CLIENT_ID = 'PY1W0T2M3B244JAWRHE414OMP3J2HJPJFNOAPRICZY0B33FG'
CLIENT_SECRET = 'ZS25SVHPO00IPA51SSN1ZOAT3YTDX1HRER5LM1H30Y4Y1A52'
VERSION = '20190909' 

In [16]:
radius = 500
LIMIT = 500
## We initiate the function to request nearby venues near with point
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
vens = getNearbyVenues(names=df_fin['Neighbourhood'],
                                   latitudes=df_fin['Latitude'],
                                   longitudes=df_fin['Longitude']
                                  )

Agincourt North
Agincourt South-Malvern West
Alderwood
Annex
Banbury-Don Mills
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods-Steeles
Bedford Park-Nortown
Beechborough-Greenbrook
Bendale
Birchcliffe-Cliffside
Black Creek
Blake-Jones
Briar Hill-Belgravia
Bridle Path-Sunnybrook-York Mills
Broadview North
Brookhaven-Amesbury
Cabbagetown-South St.James Town
Caledonia-Fairbank
Casa Loma
Centennial Scarborough
Church-Yonge Corridor
Clairlea-Birchmount
Clanton Park
Cliffcrest
Corso Italia-Davenport
Danforth
Danforth-East York
Don Valley Village
Dorset Park
Dovercourt-Wallace Emerson-Juncti
Downsview-Roding-CFB
Dufferin Grove
East End-Danforth
Edenbridge-Humber Valley
Eglinton East
Elms-Old Rexdale
Englemount-Lawrence
Eringate-Centennial-West Deane
Etobicoke West Mall
Flemingdon Park
Forest Hill North
Forest Hill South
Glenfield-Jane Heights
Greenwood-Coxwell
Henry Farm
High Park North
High Park-Swansea
Highland Creek
Hillcrest Village
Humber Heights-Westmount
Humber Summit
H

In [18]:
vens.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt North,43.74343,-79.542367,WIND Mobile,43.746810,-79.541810,Electronics Store
1,Agincourt North,43.74343,-79.542367,Shell's Food Store,43.740121,-79.540226,Grocery Store
2,Agincourt North,43.74343,-79.542367,Habitant Arena,43.747191,-79.540183,Hockey Field
3,Agincourt South-Malvern West,43.75892,-79.556175,Yashi Sports,43.762069,-79.558766,Sporting Goods Shop
4,Agincourt South-Malvern West,43.75892,-79.556175,Bad Boy Furniture - Distribution Centre,43.762881,-79.556444,Furniture / Home Store


*1.4 One Hot Encoding*

In [19]:
t_1hot = pd.get_dummies(vens[['Venue Category']], prefix="", prefix_sep="")
t_1hot['Neighborhood'] = vens['Neighborhood']
t_1hot.head()
#We place neighborhood column to the first position
temp = t_1hot['Accessories Store']
t_1hot['Accessories Store'] = t_1hot['Neighborhood']
t_1hot['Neighborhood'] = temp
t_1hot = t_1hot.rename(columns={'Accessories Store': 'Neighborhood','Neighborhood': 'Yoga Studio'})
t_1hot.shape

(2108, 291)

In [20]:
# Next we want to see the frequency of occuring of speciic type of venue in neighborhoods
t_1hotgroup = t_1hot.groupby('Neighborhood').mean().reset_index()
t_1hotgroup.head()

,Neighborhood,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt North,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt South-Malvern West,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alderwood,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Annex,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Banbury-Don Mills,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Next we display the frequency with showing top-5 venues in the first 15 neighborhoods
num_top_venues = 5
k = 0
for neigh in t_1hotgroup['Neighborhood']:
    print(neigh)
    temp = t_1hotgroup[t_1hotgroup['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    k = k + 1
    if k == 15:
        break

Agincourt North
                     venue  freq
0            Grocery Store  0.33
1        Electronics Store  0.33
2             Hockey Field  0.33
3  New American Restaurant  0.00
4             Optical Shop  0.00


Agincourt South-Malvern West
                     venue  freq
0   Furniture / Home Store   0.5
1      Sporting Goods Shop   0.5
2       African Restaurant   0.0
3               Nail Salon   0.0
4  North Indian Restaurant   0.0


Alderwood
                  venue  freq
0                  Park  0.33
1           Coffee Shop  0.33
2   Japanese Restaurant  0.33
3    African Restaurant  0.00
4  Other Great Outdoors  0.00


Annex
                        venue  freq
0                        Park  0.18
1              History Museum  0.18
2                  Steakhouse  0.09
3               Historic Site  0.09
4  Modern European Restaurant  0.09


Banbury-Don Mills
           venue  freq
0    Coffee Shop  0.13
1       Pharmacy  0.10
2           Park  0.07
3  Grocery Store  0.07
4     

In [22]:
#Next we want to add the column price...
t_1hotgroup['Price'] = df_fin['Home Prices'][2:138]

In [23]:
t_1hotgroup.dropna(axis='rows',how='any',inplace=True)
#We remove NaN values and place column with Price next to column with name of neighborhoods
t_1hotgroup.head()
temp = t_1hotgroup['African Restaurant']
t_1hotgroup['African Restaurant'] = t_1hotgroup['Price']
t_1hotgroup['Price'] = temp
t_1hotgroup = t_1hotgroup.rename(columns={'African Restaurant': 'Price','Price': 'African Restaurant'})
t_1hotgroup.shape

(132, 292)

In [24]:
#With this we finish our data preparation section
t_1hotgroup.head()

,Neighborhood,Price,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit,African Restaurant
1,Agincourt South-Malvern West,317508,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alderwood,251119,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Annex,414216,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Banbury-Don Mills,392271,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Bathurst Manor,233832,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**2. Feature selection with CHI-Square**

First of all, we are going to use the Chi-Square to measure the impact of each type of venue on the price

In [25]:
from scipy import stats
data_f = t_1hotgroup.copy()
yaxis = data_f ['Price']
feats = data_f.copy()
feats.drop(['Neighborhood','Price'],axis=1,inplace=True)
data_f.drop(['Neighborhood'],axis=1,inplace=True)

In [26]:
from sklearn.feature_selection import chi2 as f_r

In [27]:
a1st=[]
a2st=[]
a3st=[]
cols = feats.columns
for col in cols:
    try:
        a = data_f[col].to_frame()
    except:
        continue
    a['Price'] = data_f['Price']
    a.drop(a[a[col]==0].index,axis=0,inplace=True)
    c_1 = a[col]
    d_1 = a['Price']
    c_2 = []
    d_2 = []
    for k in d_1:
        k = float(k)
        d_2.append(k)
    for k in c_1:
        k = float(k)
        c_2.append(k)
    c_2 = np.array(c_2).reshape(-1, 1)
    d_2 = np.array(d_2)
    try:
        chi = f_r(c_2,d_2)
        print('Feature: '+col+' Chi-Square: '+str(chi[0])+' p-value: '+str(chi[1]))
        a1st.append(col)
        a2st.append(chi[0][0])
        a3st.append(chi[1][0])
    except:
        print('Can not calculate Chi-Square!')
        continue

Feature: American Restaurant Chi-Square: [0.89946607] p-value: [0.9999995]
Feature: Amphitheater Chi-Square: [nan] p-value: [nan]
Feature: Animal Shelter Chi-Square: [nan] p-value: [nan]
Feature: Antique Shop Chi-Square: [0.04828881] p-value: [0.97614474]
Feature: Argentinian Restaurant Chi-Square: [nan] p-value: [nan]
Feature: Art Gallery Chi-Square: [0.01233196] p-value: [0.99998107]
Feature: Art Museum Chi-Square: [nan] p-value: [nan]
Feature: Arts & Crafts Store Chi-Square: [0.01584799] p-value: [0.99996877]
Feature: Asian Restaurant Chi-Square: [0.58367541] p-value: [0.99998616]
Feature: Athletics & Sports Chi-Square: [0.36947123] p-value: [0.9990846]
Feature: Auto Dealership Chi-Square: [nan] p-value: [nan]
Feature: Auto Garage Chi-Square: [nan] p-value: [nan]
Feature: Automotive Shop Chi-Square: [nan] p-value: [nan]
Feature: BBQ Joint Chi-Square: [0.15470361] p-value: [0.99998958]
Feature: Bagel Shop Chi-Square: [0.17422127] p-value: [0.99641917]
Feature: Bakery Chi-Square: [1.3

In [28]:
featstbl = pd.DataFrame({'Name of venue':a1st,'Chi-Square':a2st,'P-value':a3st})
featstbl.head()

,Name of venue,Chi-Square,P-value
0,American Restaurant,0.899466,1.000000
1,Amphitheater,NaN,NaN
2,Animal Shelter,NaN,NaN
3,Antique Shop,0.048289,0.976145
4,Argentinian Restaurant,NaN,NaN


In [29]:
#Now we got table with features and corresponding to them F-test and p-values
#We should start with droping columns, where it wasn't possible to calculate those values - get rid of NaN
featstbl.dropna(axis='rows',how='any',inplace=True)

In [30]:
featstbl.to_csv('ChiSquareTest.csv') #We convert frame to csv, so we can explore data outside of notebook.

In [31]:
#Then we can get rid of the most obvious not-influencing venues with Chi-Square score less than 0.5
#Let's also sort our values in descending order
featstbl.drop(featstbl[featstbl['Chi-Square']<0.5].index,inplace=True)
featstbl.sort_values(by=['Chi-Square'],inplace=True,ascending=False)
featstbl

,Name of venue,Chi-Square,P-value
202,Park,10.651422,1.000000
209,Pizza Place,4.422612,1.000000
229,Sandwich Place,3.837578,1.000000
210,Playground,3.836209,0.974466
59,Coffee Shop,3.655064,1.000000
45,Café,3.505373,1.000000
223,Restaurant,3.426532,1.000000
96,Fast Food Restaurant,3.142190,1.000000
69,Convenience Store,2.404999,1.000000
148,Indian Restaurant,2.154149,0.999998


In [46]:
goodf = featstbl['Name of venue'].values
print('Features that affect the price of property the most:')
print(goodf)

Features that affect the price of property the most:
['Park' 'Pizza Place' 'Sandwich Place' 'Playground' 'Coffee Shop' 'Café'
 'Restaurant' 'Fast Food Restaurant' 'Convenience Store'
 'Indian Restaurant' 'Furniture / Home Store' 'Italian Restaurant'
 'Japanese Restaurant' 'Hotel' 'Sporting Goods Shop' 'Food & Drink Shop'
 'Discount Store' 'Bakery' 'Pharmacy' 'Chinese Restaurant' 'Flower Shop'
 'Grocery Store' 'Pool' 'Bank' 'Tea Room' 'Electronics Store'
 'American Restaurant' 'Bar' 'Theater' 'Skating Rink' 'Home Service'
 'Seafood Restaurant' 'Burger Joint' 'Dog Run' 'Asian Restaurant'
 'Gym / Fitness Center' 'Vietnamese Restaurant' 'Dessert Shop']


**3.Test of hypothesis and features in task of classification**

*Let's imagine that we have some neighborhoods with information about their venues but without information on housing price. In this section we will convert all continuous values into categorical values ('High','Medium' and 'Low' price) and then try to prove them by classification of unknown neighborhoods.*

In [33]:
#Let's copy our dataset from section one
data_sec = t_1hotgroup.copy()
data_sec.head()

,Neighborhood,Price,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit,African Restaurant
1,Agincourt South-Malvern West,317508,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alderwood,251119,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Annex,414216,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Banbury-Don Mills,392271,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Bathurst Manor,233832,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
#Then let's select only features that have been proven to be good
df_uno = data_sec[['Neighborhood','Price']]
df_uno[goodf] = data_sec[goodf]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [35]:
df_uno

,Neighborhood,Price,Park,Pizza Place,Sandwich Place,Playground,Coffee Shop,Café,Restaurant,Fast Food Restaurant,...,Theater,Skating Rink,Home Service,Seafood Restaurant,Burger Joint,Dog Run,Asian Restaurant,Gym / Fitness Center,Vietnamese Restaurant,Dessert Shop
1,Agincourt South-Malvern West,317508,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
2,Alderwood,251119,0.333333,0.000000,0.00,0.0,0.333333,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
3,Annex,414216,0.181818,0.000000,0.00,0.0,0.000000,0.090909,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
4,Banbury-Don Mills,392271,0.066667,0.066667,0.00,0.0,0.133333,0.033333,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
5,Bathurst Manor,233832,0.500000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Wychwood,426850,0.250000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.250000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
132,Yonge-Eglinton,256488,0.166667,0.166667,0.00,0.0,0.000000,0.000000,0.00,0.333333,...,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.0,0.00,0.00,0.0
133,Yonge-St.Clair,491024,0.020000,0.020000,0.02,0.0,0.110000,0.030000,0.01,0.020000,...,0.0,0.0,0.0,0.010000,0.030000,0.000000,0.0,0.01,0.01,0.0
134,York University Heights,529278,0.285714,0.000000,0.00,0.0,0.000000,0.285714,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.0,0.00,0.00,0.0


In [36]:
df_uno['Price'] = df_uno['Price'].astype(int)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
catvals = []
temp = df_uno['Price'].values
for c in temp:
    if c < 260000:
        catvals.append('Low')
    elif c > 450000:
        catvals.append('High')
    else:
        catvals.append('Medium')
print(catvals)

['Medium', 'Low', 'Medium', 'Medium', 'Low', 'Medium', 'High', 'High', 'High', 'High', 'Medium', 'High', 'Medium', 'High', 'High', 'High', 'Medium', 'High', 'High', 'High', 'Medium', 'Medium', 'Medium', 'Low', 'Medium', 'Medium', 'Medium', 'High', 'High', 'Medium', 'High', 'High', 'High', 'Medium', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'Medium', 'Low', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'Medium', 'High', 'Medium', 'High', 'High', 'High', 'High', 'Medium', 'Low', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'Medium', 'High', 'Medium', 'High', 'High', 'High', 'High', 'Medium', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'Medium', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'High', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'High', 'Medium', 'Medium', 'Medium', 'Medium', 'M

In [38]:
df_uno['Price'] = catvals
df_uno

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,Price,Park,Pizza Place,Sandwich Place,Playground,Coffee Shop,Café,Restaurant,Fast Food Restaurant,...,Theater,Skating Rink,Home Service,Seafood Restaurant,Burger Joint,Dog Run,Asian Restaurant,Gym / Fitness Center,Vietnamese Restaurant,Dessert Shop
1,Agincourt South-Malvern West,Medium,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
2,Alderwood,Low,0.333333,0.000000,0.00,0.0,0.333333,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
3,Annex,Medium,0.181818,0.000000,0.00,0.0,0.000000,0.090909,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
4,Banbury-Don Mills,Medium,0.066667,0.066667,0.00,0.0,0.133333,0.033333,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
5,Bathurst Manor,Low,0.500000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Wychwood,Medium,0.250000,0.000000,0.00,0.0,0.000000,0.000000,0.00,0.250000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.0
132,Yonge-Eglinton,Low,0.166667,0.166667,0.00,0.0,0.000000,0.000000,0.00,0.333333,...,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.0,0.00,0.00,0.0
133,Yonge-St.Clair,High,0.020000,0.020000,0.02,0.0,0.110000,0.030000,0.01,0.020000,...,0.0,0.0,0.0,0.010000,0.030000,0.000000,0.0,0.01,0.01,0.0
134,York University Heights,High,0.285714,0.000000,0.00,0.0,0.000000,0.285714,0.00,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.142857,0.0,0.00,0.00,0.0


In [39]:
#Now let's take first 20 neighborhoods for testing purpose and the rest for training purpose
test_df = df_uno[0:20]
train_df = df_uno[20:df_uno.shape[0]]
y_train = train_df['Price']
x_train = train_df.drop(['Neighborhood','Price'],axis=1)
names_test = test_df['Neighborhood']
y_test = test_df['Price']
x_test = test_df.drop(['Neighborhood','Price'],axis=1)

*As we don't know what classificator will preferm better, we will try to do it with some of them:*
1. kNN
2. SVM
3. D3

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
model_knn = KNeighborsClassifier(n_neighbors = 24).fit(x_train,y_train)
model_d3 = DecisionTreeClassifier(random_state=0,criterion="entropy", max_depth = 4).fit(x_train,y_train)
model_SVC = svm.SVC(kernel='rbf', gamma='auto').fit(x_train,y_train)

In [41]:
knnpred=model_knn.predict(x_test)
d3pred=model_d3.predict(x_test)
svcpred=model_SVC.predict(x_test)

In [42]:
y_test = np.array(y_test)

In [43]:
tempvar = 0
totnum = 0
for i in knnpred:
    if i == y_test[totnum]:
        tempvar = tempvar + 1
    totnum = totnum +1
print('Accuracy of kNN is ' + str(tempvar/totnum * 100) +'%')

Accuracy of kNN is 55.00000000000001%


In [44]:
tempvar = 0
totnum = 0
for i in d3pred:
    if i == y_test[totnum]:
        tempvar = tempvar + 1
    totnum = totnum +1
print('Accuracy of d3 is ' + str(tempvar/totnum * 100) +'%')

Accuracy of d3 is 65.0%


In [45]:
tempvar = 0
totnum = 0
for i in svcpred:
    if i == y_test[totnum]:
        tempvar = tempvar + 1
    totnum = totnum +1
print('Accuracy of SVC is ' + str(tempvar/totnum * 100) +'%')

Accuracy of SVC is 55.00000000000001%
